In [114]:
# install.packages("stargazer")
#install.packages("mvtnorm")
#install.packages("emmeans")

Installing package into ‘/home/angela/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
Warning message:
“package ‘mvtnorm’ is not available (for R version 3.4.4)”Installing package into ‘/home/angela/R/x86_64-pc-linux-gnu-library/3.4’
(as ‘lib’ is unspecified)
Warning message:
“dependency ‘mvtnorm’ is not available”Warning message in install.packages("emmeans"):
“installation of package ‘emmeans’ had non-zero exit status”

In [112]:
library("reshape2")
library("stargazer")
#library("emmeans")

ERROR: Error in library("emmeans"): there is no package called ‘emmeans’


In [2]:
absPath = '/home/angela/padding_EBI/'

In [ ]:
file_scores <- "scores_final.csv"
file_accuracy <- "accu_final.csv"

# Task 1

In [6]:
task <- "task1/"

path_scores_task1 <- paste0(absPath, "data/results/", task, file_scores)
path_accu_task1 <- paste0(absPath, "data/results/", task, file_accuracy)

scores_task1 <- read.csv(file=path_scores_task1, header=TRUE, sep=",")
accu_task1 <- read.csv(file=path_accu_task1, header=TRUE, sep=",")

## Scores

In [13]:
scores_task1[0:5,]
scores_task1$X <- NULL

X,enz_type,index,type_padding,architecture,variable,value
<int>,<fct>,<int>,<fct>,<fct>,<fct>,<dbl>
0,0,0,post_padding,1_conv,f1-score,0.8548043
1,1,0,post_padding,1_conv,f1-score,0.8642715
2,micro avg,0,post_padding,1_conv,f1-score,0.8596974
3,macro avg,0,post_padding,1_conv,f1-score,0.8595379
4,weighted avg,0,post_padding,1_conv,f1-score,0.8600099


In [16]:
#filtering 
scores_task1 <- scores_task1[scores_task1$enz_type %in% c(0,1),]

sapply(scores_task1, class)

enz_type        index type_padding architecture     variable        value 
    "factor"    "integer"     "factor"     "factor"     "factor"    "numeric"

In [62]:
summary(scores_task1$enz_type)

0            1    macro avg    micro avg weighted avg 
         960          960            0            0            0

In [31]:
# habria que hacer un unmelt de variable-value
scores_task1_dcasted <- dcast(data = scores_task1,formula = enz_type+index+type_padding+architecture~variable,
                              fun.aggregate = sum,value.var = "value")

In [33]:
#tengo que cambiar el nombre de f1-score
colnames(scores_task1_dcasted) <- c("enz_type", "index", "type_padding", "architecture", "f1_score",
                                    "precision", "recall")
scores_task1_dcasted[0:5,]

enz_type,index,type_padding,architecture,f1_score,precision,recall
<fct>,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
0,0,aug_padding,1_conv,0.7512593,0.6471795,0.8952308
0,0,aug_padding,bi_lstm,0.7167895,0.5736383,0.9551457
0,0,aug_padding,only_denses,0.8760426,0.8434343,0.9112736
0,0,aug_padding,stack_conv,0.7811460,0.6844007,0.9097457
0,0,ext_padding,1_conv,0.8403179,0.7971213,0.8884645


In [122]:
scores_task1_dcasted$architecture <- relevel(scores_task1_dcasted$architecture, "only_denses")
scores_task1_dcasted$type_padding <- relevel(scores_task1_dcasted$type_padding, "post_padding")

In [123]:
lm_f1_task1_additive <- lm(f1_score~architecture+enz_type+type_padding, data=scores_task1_dcasted)

In [124]:
summary(lm_f1_task1_additive)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding, 
    data = scores_task1_dcasted)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.72626 -0.01819  0.00412  0.02333  0.09002 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.915552   0.006112 149.794  < 2e-16 ***
architecture1_conv       -0.063287   0.004990 -12.682  < 2e-16 ***
architecturebi_lstm      -0.143906   0.004990 -28.836  < 2e-16 ***
architecturestack_conv   -0.052351   0.004990 -10.490  < 2e-16 ***
enz_type1                 0.024232   0.003529   6.867 1.58e-11 ***
type_paddingaug_padding  -0.073154   0.007058 -10.365  < 2e-16 ***
type_paddingext_padding  -0.045384   0.007058  -6.431 2.52e-10 ***
type_paddingmid_padding   0.005152   0.007058   0.730   0.4657    
type_paddingpre_padding  -0.011706   0.007058  -1.659   0.0977 .  
type_paddingrnd_padding  -0.052515   0.007058  -7.441 3.30e-13 ***
type_paddingstrf_padding -0.010308   0.

In [130]:
soa_arch <- c("stack_conv", "bi_lstm")
soa_pad <- c("post_padding", "pre_padding", "mid_padding", "ext_padding")
lm_f1_task1_additive_soa <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, 
                               data=subset(scores_task1_dcasted, architecture %in% soa_arch & type_padding %in% soa_pad))

In [131]:
summary(lm_f1_task1_additive_soa)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task1_dcasted, 
    architecture %in% soa_arch & type_padding %in% soa_pad))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.63387 -0.01356  0.00976  0.02012  0.09387 

Coefficients:
                                                Estimate Std. Error t value
(Intercept)                                     0.776724   0.014281  54.387
architecturestack_conv                          0.080073   0.019042   4.205
enz_type1                                       0.044692   0.009521   4.694
type_paddingext_padding                        -0.142850   0.019042  -7.502
type_paddingmid_padding                         0.020414   0.019042   1.072
type_paddingpre_padding                        -0.014794   0.019042  -0.777
architecturestack_conv:type_paddingext_padding  0.138917   0.026929   5.159
architecturestack_conv:type_paddingmid_padding -0.025990   0.026929  -0

In [142]:
# interactions non significant
soa2_pad <- c("post_padding", "strf_padding", "rnd_padding", "zoom_padding")
lm_f1_task1_additive_soa2 <- lm(f1_score~architecture+enz_type+type_padding, 
                               data=subset(scores_task1_dcasted, architecture %in% soa_arch & type_padding %in% soa2_pad))

In [143]:
summary(lm_f1_task1_additive_soa2)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding, 
    data = subset(scores_task1_dcasted, architecture %in% soa_arch & 
        type_padding %in% soa2_pad))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.125632 -0.013578  0.002155  0.016052  0.049521 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.777703   0.004489 173.256  < 2e-16 ***
architecturestack_conv    0.075906   0.003665  20.711  < 2e-16 ***
enz_type1                 0.046901   0.003665  12.797  < 2e-16 ***
type_paddingrnd_padding  -0.059636   0.005183 -11.506  < 2e-16 ***
type_paddingstrf_padding -0.032954   0.005183  -6.358 2.21e-09 ***
type_paddingzoom_padding -0.010754   0.005183  -2.075   0.0397 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.02318 on 154 degrees of freedom
Multiple R-squared:  0.8293,	Adjusted R-squared:  0.8238 
F-statistic: 149.6 on 5 and 154 DF,  p

In [145]:
lm_f1_task1_additive_soa3 <- lm(f1_score~enz_type+type_padding, 
                               data=subset(scores_task1_dcasted, architecture == "only_denses"))

In [146]:
summary(lm_f1_task1_additive_soa3)


Call:
lm(formula = f1_score ~ enz_type + type_padding, data = subset(scores_task1_dcasted, 
    architecture == "only_denses"))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.075505 -0.008432  0.002578  0.010467  0.031178 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.8956930  0.0038765 231.057  < 2e-16 ***
enz_type1                 0.0087738  0.0025843   3.395 0.000877 ***
type_paddingaug_padding  -0.0001935  0.0051687  -0.037 0.970191    
type_paddingext_padding  -0.0041699  0.0051687  -0.807 0.421071    
type_paddingmid_padding   0.0105087  0.0051687   2.033 0.043789 *  
type_paddingpre_padding  -0.0008379  0.0051687  -0.162 0.871442    
type_paddingrnd_padding  -0.0041826  0.0051687  -0.809 0.419664    
type_paddingstrf_padding  0.0300487  0.0051687   5.814 3.51e-08 ***
type_paddingzoom_padding -0.0067748  0.0051687  -1.311 0.191932    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘

In [125]:
lm_f1_task1 <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, data=scores_task1_dcasted)

In [126]:
summary(lm_f1_task1)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = scores_task1_dcasted)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.64410 -0.01056  0.00265  0.01684  0.10410 

Coefficients:
                                                  Estimate Std. Error t value
(Intercept)                                      0.8879640  0.0085722 103.586
architecture1_conv                              -0.0272441  0.0119379  -2.282
architecturebi_lstm                             -0.1010102  0.0119379  -8.461
architecturestack_conv                          -0.0209371  0.0119379  -1.754
enz_type1                                        0.0242318  0.0029845   8.119
type_paddingaug_padding                         -0.0001935  0.0119379  -0.016
type_paddingext_padding                         -0.0041699  0.0119379  -0.349
type_paddingmid_padding                          0.0105087  0.0119379   0.880
type_paddingpre_padding                       

In [95]:
lm_f1_task1 <- lm(f1_score~architecture+enz_type+type_padding, data=scores_task1_dcasted)
summary(lm_f1_task1)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding, 
    data = scores_task1_dcasted)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.72626 -0.01819  0.00412  0.02333  0.09002 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.779111   0.006112 127.471  < 2e-16 ***
architecturebi_lstm      -0.080619   0.004990 -16.155  < 2e-16 ***
architectureonly_denses   0.063287   0.004990  12.682  < 2e-16 ***
architecturestack_conv    0.010937   0.004990   2.192  0.02878 *  
enz_type1                 0.024232   0.003529   6.867 1.58e-11 ***
type_paddingext_padding   0.027770   0.007058   3.935 9.26e-05 ***
type_paddingmid_padding   0.078306   0.007058  11.095  < 2e-16 ***
type_paddingpost_padding  0.073154   0.007058  10.365  < 2e-16 ***
type_paddingpre_padding   0.061448   0.007058   8.707  < 2e-16 ***
type_paddingrnd_padding   0.020639   0.007058   2.924  0.00358 ** 
type_paddingstrf_padding  0.062846   0.

In [111]:
# stargazer(lm_f1_task1, title="Linear models on F1 score", align=TRUE)

In [147]:
soa_pad <- c("post_padding", "strf_padding")
lm_f1_task1_soa4 <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, 
                               data=subset(scores_task1_dcasted, type_padding %in% soa_pad))

In [148]:
summary(lm_f1_task1_soa4)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task1_dcasted, 
    type_padding %in% soa_pad))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.133528 -0.009068 -0.000109  0.012975  0.037458 

Coefficients:
                                                 Estimate Std. Error t value
(Intercept)                                      0.886609   0.004840 183.167
architecture1_conv                              -0.027244   0.006454  -4.221
architecturebi_lstm                             -0.101010   0.006454 -15.651
architecturestack_conv                          -0.020937   0.006454  -3.244
enz_type1                                        0.026942   0.003227   8.349
type_paddingstrf_padding                         0.030049   0.006454   4.656
architecture1_conv:type_paddingstrf_padding     -0.035423   0.009127  -3.881
architecturebi_lstm:type_paddingstrf_padding    -0.068014   0.009127  -7.452
archi

In [187]:
soa_pad <- c("post_padding", "strf_padding", "aug_padding")
lm_f1_task1_soa5 <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, 
                               data=subset(scores_task1_dcasted, type_padding %in% soa_pad))
summary(lm_f1_task1_soa5)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task1_dcasted, 
    type_padding %in% soa_pad))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.173831 -0.012287  0.003771  0.013166  0.081870 

Coefficients:
                                                  Estimate Std. Error t value
(Intercept)                                      0.8941772  0.0071540 124.989
architecture1_conv                              -0.0272441  0.0097204  -2.803
architecturebi_lstm                             -0.1010102  0.0097204 -10.392
architecturestack_conv                          -0.0209371  0.0097204  -2.154
enz_type1                                        0.0118053  0.0039684   2.975
type_paddingaug_padding                         -0.0001935  0.0097204  -0.020
type_paddingstrf_padding                         0.0300487  0.0097204   3.091
architecture1_conv:type_paddingaug_padding      -0.1165799  0.0137468  -8.

In [191]:
soa_pad <- c("post_padding", "strf_padding", "aug_padding")
scores_task1_dcasted_aug <- scores_task1_dcasted
scores_task1_dcasted_aug$type_padding <- relevel(scores_task1_dcasted_aug$type_padding, "aug_padding")
lm_f1_task2_soa5 <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, 
                               data=subset(scores_task1_dcasted_aug, type_padding %in% soa_pad))
summary(lm_f1_task2_soa5)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task1_dcasted_aug, 
    type_padding %in% soa_pad))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.173831 -0.012287  0.003771  0.013166  0.081870 

Coefficients:
                                                  Estimate Std. Error t value
(Intercept)                                      0.8939838  0.0071540 124.962
architecture1_conv                              -0.1438240  0.0097204 -14.796
architecturebi_lstm                             -0.1873464  0.0097204 -19.273
architecturestack_conv                          -0.1098636  0.0097204 -11.302
enz_type1                                        0.0118053  0.0039684   2.975
type_paddingpost_padding                         0.0001935  0.0097204   0.020
type_paddingstrf_padding                         0.0302421  0.0097204   3.111
architecture1_conv:type_paddingpost_padding      0.1165799  0.0137468 

### Accuracy

In [74]:
accu_task1[0:5,]
accu_task2$X <- NULL

index,variable,value,architecture
<int>,<fct>,<dbl>,<fct>
0,post_padding,0.8596974,1_conv
1,post_padding,0.8820495,1_conv
2,post_padding,0.8786107,1_conv
3,post_padding,0.8772352,1_conv
4,post_padding,0.8693260,1_conv


In [11]:
#lo hago con la accuracy pero sin el tipo de enzima

In [75]:
#filtering 
sapply(accu_task1, class)

index     variable        value architecture 
   "integer"     "factor"    "numeric"     "factor"

In [77]:
#cambio el nombre a las columnas
colnames(accu_task1) <- c("index", "type_padding", "accuracy", "architecture")
accu_task1[0:5,]

index,type_padding,accuracy,architecture
<int>,<fct>,<dbl>,<fct>
0,post_padding,0.8596974,1_conv
1,post_padding,0.8820495,1_conv
2,post_padding,0.8786107,1_conv
3,post_padding,0.8772352,1_conv
4,post_padding,0.8693260,1_conv


In [93]:
lm_acc_task1 <- lm(accuracy~architecture+type_padding+architecture:type_padding, data=accu_task1)

In [94]:
summary(lm_acc_task1)


Call:
lm(formula = accuracy ~ architecture + type_padding + architecture:type_padding, 
    data = accu_task1)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.129711 -0.007806  0.001840  0.009964  0.067302 

Coefficients:
                                                   Estimate Std. Error t value
(Intercept)                                       0.7579829  0.0064654 117.237
architecturebi_lstm                              -0.0397180  0.0091434  -4.344
architectureonly_denses                           0.1425329  0.0091434  15.589
architecturestack_conv                            0.0325113  0.0091434   3.556
type_paddingext_padding                           0.0845205  0.0091434   9.244
type_paddingmid_padding                           0.1102427  0.0091434  12.057
type_paddingpost_padding                          0.1152289  0.0091434  12.602
type_paddingpre_padding                           0.1005109  0.0091434  10.993
type_paddingrnd_padding                         

In [110]:
#stargazer(lm_acc_task1, title="Linear models on F1 score", align=TRUE)

# Task 2

In [166]:
task <- "task2/"

path_scores_task2 <- paste0(absPath, "data/results/", task, file_scores)
path_accu_task2 <- paste0(absPath, "data/results/", task, file_accuracy)

scores_task2 <- read.csv(file=path_scores_task2, header=TRUE, sep=",")
accu_task2 <- read.csv(file=path_accu_task2, header=TRUE, sep=",")

## Scores

In [167]:
scores_task2[0:5,]
dim(scores_task2)
scores_task2$X <- NULL

X,enz_type,index,type_padding,architecture,variable,value
<int>,<fct>,<int>,<fct>,<fct>,<fct>,<dbl>
0,1,0,post_padding,1_conv,f1-score,0.4609665
1,2,0,post_padding,1_conv,f1-score,0.6056475
2,3,0,post_padding,1_conv,f1-score,0.5493562
3,4,0,post_padding,1_conv,f1-score,0.4986877
4,5,0,post_padding,1_conv,f1-score,0.6060606


[1] 9600    7

In [168]:
#filtering 
scores_task2 <- scores_task2[scores_task2$enz_type %in% c(1,2,3,4,5,6,7),]
dim(scores_task2)
sapply(scores_task2, class)

[1] 6720    6

enz_type        index type_padding architecture     variable        value 
    "factor"    "integer"     "factor"     "factor"     "factor"    "numeric"

In [169]:
# habria que hacer un unmelt de variable-value
scores_task2_dcasted <- dcast(data = scores_task2, formula = enz_type+index+type_padding+architecture~variable,
                              fun.aggregate = sum,value.var = "value")

In [170]:
#tengo que cambiar el nombre de f1-score
colnames(scores_task2_dcasted) <- c("enz_type", "index", "type_padding", "architecture", "f1_score",
                                    "precision", "recall")
scores_task2_dcasted[0:5,]

enz_type,index,type_padding,architecture,f1_score,precision,recall
<fct>,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,0,aug_padding,1_conv,0.4514306,0.4031546,0.51284109
1,0,aug_padding,bi_lstm,0.1046196,0.3407080,0.06179775
1,0,aug_padding,only_denses,0.4247295,0.3453997,0.55136437
1,0,aug_padding,stack_conv,0.4891587,0.5320755,0.45264848
1,0,ext_padding,1_conv,0.3833866,0.2678571,0.67415730


In [171]:
scores_task2_dcasted[0:5,]

enz_type,index,type_padding,architecture,f1_score,precision,recall
<fct>,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>
1,0,aug_padding,1_conv,0.4514306,0.4031546,0.51284109
1,0,aug_padding,bi_lstm,0.1046196,0.3407080,0.06179775
1,0,aug_padding,only_denses,0.4247295,0.3453997,0.55136437
1,0,aug_padding,stack_conv,0.4891587,0.5320755,0.45264848
1,0,ext_padding,1_conv,0.3833866,0.2678571,0.67415730


In [172]:
scores_task2_dcasted$architecture <- relevel(scores_task2_dcasted$architecture, "only_denses")
scores_task2_dcasted$type_padding <- relevel(scores_task2_dcasted$type_padding, "post_padding")

In [173]:
lm_f1_task2_additive <- lm(f1_score~architecture+enz_type+type_padding, 
                           data=scores_task2_dcasted)

In [174]:
summary(lm_f1_task2_additive)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding, 
    data = scores_task2_dcasted)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.51449 -0.05887 -0.00777  0.04113  0.40756 

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.4422464  0.0090377  48.934  < 2e-16 ***
architecture1_conv        0.0206864  0.0061998   3.337 0.000862 ***
architecturebi_lstm      -0.4157025  0.0061998 -67.051  < 2e-16 ***
architecturestack_conv    0.0101555  0.0061998   1.638 0.101557    
enz_type2                 0.2559840  0.0082016  31.212  < 2e-16 ***
enz_type3                 0.0938845  0.0082016  11.447  < 2e-16 ***
enz_type4                 0.0039705  0.0082016   0.484 0.628352    
enz_type5                 0.1226206  0.0082016  14.951  < 2e-16 ***
enz_type6                 0.0426076  0.0082016   5.195 2.23e-07 ***
enz_type7                 0.0722387  0.0082016   8.808  < 2e-16 ***
type_paddingaug_padding  -0.

In [175]:
soa_arch <- c("stack_conv", "bi_lstm")
soa_pad <- c("post_padding", "pre_padding", "mid_padding", "ext_padding")
lm_f1_task2_additive_soa <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture,
                               data=subset(scores_task2_dcasted, architecture %in% soa_arch &
                                          type_padding %in% soa_pad))
summary(lm_f1_task2_additive_soa)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task2_dcasted, 
    architecture %in% soa_arch & type_padding %in% soa_pad))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.20633 -0.06346 -0.01382  0.05131  0.36483 

Coefficients:
                                                Estimate Std. Error t value
(Intercept)                                     0.020241   0.016375   1.236
architecturestack_conv                          0.443730   0.017505  25.349
enz_type2                                       0.344084   0.016375  21.013
enz_type3                                       0.109937   0.016375   6.714
enz_type4                                      -0.008815   0.016375  -0.538
enz_type5                                       0.065530   0.016375   4.002
enz_type6                                       0.057103   0.016375   3.487
enz_type7                                       0.059785   0.016375   3

In [182]:
soa_pad <- c("post_padding", "strf_padding", "rnd_padding", "zoom_padding")
lm_f1_task2_additive_soa2 <- lm(f1_score~architecture+enz_type+type_padding+type_padding*architecture,
                               data=subset(scores_task2_dcasted, architecture %in% soa_arch &
                                          type_padding %in% soa_pad))

In [183]:
summary(lm_f1_task2_additive_soa2)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding * architecture, data = subset(scores_task2_dcasted, 
    architecture %in% soa_arch & type_padding %in% soa_pad))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.40811 -0.04638 -0.00637  0.04437  0.37064 

Coefficients:
                                                  Estimate Std. Error t value
(Intercept)                                      0.0405106  0.0150445   2.693
architecturestack_conv                           0.4437296  0.0160832  27.590
enz_type2                                        0.3422050  0.0150445  22.746
enz_type3                                        0.0662313  0.0150445   4.402
enz_type4                                       -0.0078110  0.0150445  -0.519
enz_type5                                        0.0633978  0.0150445   4.214
enz_type6                                       -0.0003121  0.0150445  -0.021
enz_type7                                        0.02

In [185]:
lm_f1_task2_additive_soa3 <- lm(f1_score~enz_type+type_padding, 
                                data=subset(scores_task2_dcasted, architecture=="only_denses"))

summary(lm_f1_task2_additive_soa3 )


Call:
lm(formula = f1_score ~ enz_type + type_padding, data = subset(scores_task2_dcasted, 
    architecture == "only_denses"))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.49560 -0.03289  0.00192  0.03583  0.43419 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.43554    0.01546  28.170  < 2e-16 ***
enz_type2                 0.15236    0.01546   9.854  < 2e-16 ***
enz_type3                 0.10727    0.01546   6.938 1.13e-11 ***
enz_type4                 0.00524    0.01546   0.339 0.734809    
enz_type5                 0.16644    0.01546  10.765  < 2e-16 ***
enz_type6                 0.05805    0.01546   3.754 0.000192 ***
enz_type7                 0.02208    0.01546   1.428 0.153929    
type_paddingaug_padding   0.02269    0.01653   1.373 0.170354    
type_paddingext_padding   0.03393    0.01653   2.053 0.040543 *  
type_paddingmid_padding   0.03373    0.01653   2.041 0.041749 *  
type_paddingpre_padding   

In [186]:
soa_pad <- c("post_padding", "strf_padding")
lm_f1_task2_soa4 <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, 
                               data=subset(scores_task2_dcasted, type_padding %in% soa_pad))
summary(lm_f1_task2_soa4)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task2_dcasted, 
    type_padding %in% soa_pad))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.51926 -0.05773 -0.00332  0.04109  0.38064 

Coefficients:
                                                Estimate Std. Error t value
(Intercept)                                      0.42715    0.01598  26.736
architecture1_conv                               0.04128    0.01708   2.417
architecturebi_lstm                             -0.39870    0.01708 -23.343
architecturestack_conv                           0.04503    0.01708   2.636
enz_type2                                        0.25161    0.01598  15.749
enz_type3                                        0.09749    0.01598   6.102
enz_type4                                       -0.01585    0.01598  -0.992
enz_type5                                        0.10563    0.01598   6.612
enz_type6               

In [193]:
soa_pad <- c("post_padding", "strf_padding", "aug_padding")
scores_task2_dcasted_aug <- scores_task2_dcasted
scores_task2_dcasted_aug$type_padding <- relevel(scores_task2_dcasted_aug$type_padding, "aug_padding")
lm_f1_task2_soa5 <- lm(f1_score~architecture+enz_type+type_padding+type_padding:architecture, 
                               data=subset(scores_task2_dcasted_aug, type_padding %in% soa_pad))
summary(lm_f1_task2_soa5)


Call:
lm(formula = f1_score ~ architecture + enz_type + type_padding + 
    type_padding:architecture, data = subset(scores_task2_dcasted_aug, 
    type_padding %in% soa_pad))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.51669 -0.05483 -0.00686  0.03844  0.38321 

Coefficients:
                                                 Estimate Std. Error t value
(Intercept)                                      0.450800   0.014164  31.827
architecture1_conv                               0.004791   0.016355   0.293
architecturebi_lstm                             -0.442202   0.016355 -27.037
architecturestack_conv                          -0.027212   0.016355  -1.664
enz_type2                                        0.252614   0.012492  20.223
enz_type3                                        0.087289   0.012492   6.988
enz_type4                                       -0.001736   0.012492  -0.139
enz_type5                                        0.111277   0.012492   8.908
enz_type6  

## Accuracy

In [73]:
accu_task2[0:5,]

X,index,variable,value,architecture
<int>,<int>,<fct>,<dbl>,<fct>
0,0,post_padding,0.5472170,1_conv
1,1,post_padding,0.5284553,1_conv
2,2,post_padding,0.5121951,1_conv
3,3,post_padding,0.5247029,1_conv
4,4,post_padding,0.5247029,1_conv


In [81]:
#filtering 
sapply(accu_task2, class)

index     variable        value architecture 
   "integer"     "factor"    "numeric"     "factor"

In [82]:
#cambio el nombre a las columnas
colnames(accu_task2) <- c("index", "type_padding", "accuracy", "architecture")
accu_task2[0:5,]

index,type_padding,accuracy,architecture
<int>,<fct>,<dbl>,<fct>
0,post_padding,0.5472170,1_conv
1,post_padding,0.5284553,1_conv
2,post_padding,0.5121951,1_conv
3,post_padding,0.5247029,1_conv
4,post_padding,0.5247029,1_conv


In [91]:
lm_accu_task2 <- lm(accuracy~architecture+type_padding+ type_padding:architecture, data=accu_task2)

In [92]:
summary(lm_accu_task2)


Call:
lm(formula = accuracy ~ architecture + type_padding + type_padding:architecture, 
    data = accu_task2)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.058286 -0.006551  0.000000  0.008456  0.043340 

Coefficients:
                                                  Estimate Std. Error t value
(Intercept)                                       0.547923   0.004672 117.279
architecturebi_lstm                              -0.185580   0.006607 -28.088
architectureonly_denses                          -0.012088   0.006607  -1.830
architecturestack_conv                           -0.021379   0.006607  -3.236
type_paddingext_padding                          -0.009086   0.006607  -1.375
type_paddingmid_padding                          -0.009086   0.006607  -1.375
type_paddingpost_padding                         -0.017716   0.006607  -2.681
type_paddingpre_padding                          -0.015528   0.006607  -2.350
type_paddingrnd_padding                          -0.09276

## Construyendo tablas

In [107]:
stargazer(lm_acc_task1,lm_accu_task2, title="Linear models on F1 score", align=TRUE)


% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Wed, Oct 30, 2019 - 12:29:50 PM
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{Linear models on F1 score} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{\textit{Dependent variable:}} \\ 
\cline{2-3} 
\\[-1.8ex] & \multicolumn{2}{c}{accuracy} \\ 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)}\\ 
\hline \\[-1.8ex] 
 architecturebi\_lstm & -0.040^{***} & -0.186^{***} \\ 
  & (0.009) & (0.007) \\ 
  & & \\ 
 architectureonly\_denses & 0.143^{***} & -0.012^{*} \\ 
  & (0.009) & (0.007) \\ 
  & & \\ 
 architecturestack\_conv & 0.033^{***} & -0.021^{***} \\ 
  & (0.009) & (0.007) \\ 
  & & \\ 
 type\_paddingext\_padding & 0.085^{***} & -0.009 \\ 
  & (0.009) & (0.007) \\ 
  & & \\ 
 type\_paddingmid\_padding & 0.110^{***} & -0.009 \\

In [109]:
stargazer(lm_f1_task1,lm_f1_task2, title="Linear models on F1 score", align=TRUE)


% Table created by stargazer v.5.2.2 by Marek Hlavac, Harvard University. E-mail: hlavac at fas.harvard.edu
% Date and time: Wed, Oct 30, 2019 - 12:30:37 PM
% Requires LaTeX packages: dcolumn 
\begin{table}[!htbp] \centering 
  \caption{Linear models on F1 score} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lD{.}{.}{-3} D{.}{.}{-3} } 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{2}{c}{\textit{Dependent variable:}} \\ 
\cline{2-3} 
\\[-1.8ex] & \multicolumn{2}{c}{f1\_score} \\ 
\\[-1.8ex] & \multicolumn{1}{c}{(1)} & \multicolumn{1}{c}{(2)}\\ 
\hline \\[-1.8ex] 
 architecturebi\_lstm & -0.081^{***} & -0.436^{***} \\ 
  & (0.005) & (0.006) \\ 
  & & \\ 
 architectureonly\_denses & 0.063^{***} & -0.021^{***} \\ 
  & (0.005) & (0.006) \\ 
  & & \\ 
 architecturestack\_conv & 0.011^{**} & -0.011^{*} \\ 
  & (0.005) & (0.006) \\ 
  & & \\ 
 enz\_type1 & 0.024^{***} &  \\ 
  & (0.004) &  \\ 
  & & \\ 
 enz\_type2 &  & 0.256^{***} \\ 
  &  & (0.008) \\ 
  & & \\ 
 enz\_type3 &  & 0